In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split

* Importar as bases de treino e teste para o desafio

In [48]:
data= pd.read_csv("train.csv")
data2= pd.read_csv("test.csv")

* separar a base em targuet e explicativas 

In [49]:
targuet = data["NU_NOTA_MT"]

In [50]:
# Esta parte foi colocada aqui, pois na hora de executar o modelo na base de teste as colunas não eram as mesmas da base de treino e isso dava erro
data = data[data2.columns]

In [51]:
data.shape
#(13730, 47)

(13730, 47)

In [52]:
data2.shape

(4576, 47)

<h1>Vamos escolher e tratas as features da base de treino para criar nosso modelo</h1>

In [53]:
porcentagem_nulos = pd.DataFrame(data.isnull().sum()/data.shape[0]*100,columns=['porcentagem'])

* Para tentar melhorar o modelo vamos remover as features com mais de 50% de nulos

In [54]:
porcentagem_nulos[porcentagem_nulos.porcentagem > 50].T

,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,Q027
porcentagem,68.812819,68.812819,53.699927


* baseado no Dicionario_Microdados_Enem_2016 e nas features disponiveis na base de testes vamos listar as features para gerar Dummies depois

In [55]:
fazer_dummies = [
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 #'TP_ENSINO', #mais que 50% nulo
 'IN_TREINEIRO',
 #'TP_DEPENDENCIA_ADM_ESC', #mais que 50% nulo
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 #'Q027', #mais que 50% nulo
 'Q047']

In [56]:
# criar as dimmies e atribuir na base de dados
dummies = pd.get_dummies(data[fazer_dummies])
data[dummies.columns] = dummies

dummies = pd.get_dummies(data2[fazer_dummies])
data2[dummies.columns] = dummies

In [57]:
data.shape

(13730, 124)

In [58]:
data2.shape

(4576, 124)

* Para fazer a analise, não vamos usar todas as features e temos que retirar as features que deram origem as dummies

In [59]:
# 
colunas_para_remover = ['SG_UF_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'NU_INSCRICAO',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'TP_DEPENDENCIA_ADM_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q027',
 'Q047']


In [60]:
explanatory_features = list(data.columns)
for i in colunas_para_remover:
    explanatory_features.remove(i)
explanatory_features = data[explanatory_features]
#(13730, 78)

In [61]:
explanatory_features.shape

(13730, 87)

In [62]:
explanatory_features.dtypes

NU_IDADE           int64
NU_NOTA_CN       float64
NU_NOTA_CH       float64
NU_NOTA_LC       float64
NU_NOTA_COMP1    float64
                  ...   
Q047_A             uint8
Q047_B             uint8
Q047_C             uint8
Q047_D             uint8
Q047_E             uint8
Length: 87, dtype: object

* para melhorar o modelo vamos converter as notas de float para int e assumir que os nulos são nota zero

In [63]:
teste = pd.DataFrame()
for i in explanatory_features.columns:
    if explanatory_features[i].dtype == 'float':
        teste[i]=(explanatory_features[i].fillna(explanatory_features[i].mode().mean())*100).astype(int)
teste.describe()
explanatory_features[teste.columns] = teste[teste.columns] 

C:\Users\1570 LITE\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [64]:
list(explanatory_features.columns)

['NU_IDADE',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'SG_UF_RESIDENCIA_AC',
 'SG_UF_RESIDENCIA_AL',
 'SG_UF_RESIDENCIA_AM',
 'SG_UF_RESIDENCIA_AP',
 'SG_UF_RESIDENCIA_BA',
 'SG_UF_RESIDENCIA_CE',
 'SG_UF_RESIDENCIA_DF',
 'SG_UF_RESIDENCIA_ES',
 'SG_UF_RESIDENCIA_GO',
 'SG_UF_RESIDENCIA_MA',
 'SG_UF_RESIDENCIA_MG',
 'SG_UF_RESIDENCIA_MS',
 'SG_UF_RESIDENCIA_MT',
 'SG_UF_RESIDENCIA_PA',
 'SG_UF_RESIDENCIA_PB',
 'SG_UF_RESIDENCIA_PE',
 'SG_UF_RESIDENCIA_PI',
 'SG_UF_RESIDENCIA_PR',
 'SG_UF_RESIDENCIA_RJ',
 'SG_UF_RESIDENCIA_RN',
 'SG_UF_RESIDENCIA_RO',
 'SG_UF_RESIDENCIA_RR',
 'SG_UF_RESIDENCIA_RS',
 'SG_UF_RESIDENCIA_SC',
 'SG_UF_RESIDENCIA_SE',
 'SG_UF_RESIDENCIA_SP',
 'SG_UF_RESIDENCIA_TO',
 'TP_SEXO_F',
 'TP_SEXO_M',
 'Q001_A',
 'Q001_B',
 'Q001_C',
 'Q001_D',
 'Q001_E',
 'Q001_F',
 'Q001_G',
 'Q001_H',
 'Q002_A',
 'Q002_B',
 'Q002_C',
 'Q002_D',
 'Q002_E',
 'Q002_F',
 '

* para melhorar o modelo vamos converter as notas de float para int e assumir que os nulos são nota zero e vamos salvar em uma variavel diferente para não modificar a targuet original, para não perder os valores apos a virgula, vamos multiplicar por 100

In [65]:
alvo = (targuet.fillna(targuet.mode().mean())*100).astype(int)

In [66]:
alvo

0        39940
1        45980
2        44530
3        44530
4        44530
         ...  
13725    40320
13726    45240
13727    39800
13728    38660
13729    42890
Name: NU_NOTA_MT, Length: 13730, dtype: int32

* Dividir a base em treino e teste

In [67]:
x_train, x_test, y_train, y_test = train_test_split(explanatory_features, alvo, test_size=0.33, random_state=42)

<h1>Vamos rodar o modelo normalizando e não normalizando o modelo</h1>

In [68]:
# Create linear regression object
regr2 = linear_model.LinearRegression(normalize=True,n_jobs=4)

# Train the model using the training sets
regr2.fit(x_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=4, normalize=True)

* Apos treinar o modelo, vamos voltar as notas para a ordem de grandeza corretas

In [69]:
y_test=y_test/100

* Vamos fazer as predições usando os modelos treinados

In [70]:
#normalizado
y_test_pred2 = regr2.predict(x_test)
y_test_pred2 = y_test_pred2/100


In [71]:
y_test_pred2.min()

31.68

* como as notas não podem ser negativas, então vamos substituir por zero

In [72]:
#norm_zera_neg
y_test_pred2_1 = y_test_pred2.copy()
y_test_pred2_1[y_test_pred2_1<0] = 0

* outra opção é converter para positivo

In [73]:
#norm_abs
y_test_pred2_2 = y_test_pred2.copy()
y_test_pred2_2[y_test_pred2_2<0] = y_test_pred2_2[y_test_pred2_2<0]*-1

* Vamos ver as metricas das tentativas

In [74]:
metricas= pd.DataFrame(columns=['Mean Absolute Error:','Mean Squared Error:','Root Mean Squared Error:'],
index=['normalizado','norm_zera_neg','norm_abs'],
data=[
[metrics.mean_absolute_error(y_test, y_test_pred2),metrics.mean_squared_error(y_test, y_test_pred2),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2))],
[metrics.mean_absolute_error(y_test, y_test_pred2_1),metrics.mean_squared_error(y_test, y_test_pred2_1),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2_1))],
[metrics.mean_absolute_error(y_test, y_test_pred2_2),metrics.mean_squared_error(y_test, y_test_pred2_2),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2_2))]])

# nao_normalizado	50.164493	4743.872600	68.875777
# normalizado	50.165393	4743.980423	68.876559
# nao_norm_zera_neg	48.371068	4718.357238	68.690299
# norm_zera_neg	48.368426	4718.419648	68.690754
# nao_norm_abs	50.164493	4743.872600	68.875777
# norm_abs	50.165393	4743.980423	68.876559
# nao_norm_abs_max_lim	48.371068	4718.357238	68.690299
# norm_abs_max_lim	48.368426	4718.419648	68.690754

In [75]:
metricas

,Mean Absolute Error:,Mean Squared Error:,Root Mean Squared Error:
normalizado,46.83424,4232.335007,65.056399
norm_zera_neg,46.83424,4232.335007,65.056399
norm_abs,46.83424,4232.335007,65.056399


* O melhor modelo foi o modelo em que substituimos as predições negativas por zero

<h1>Vamos aplicar o modelo na base de testes</h1>

* Vamos selecionar as features da mesma forma ue foi feita na base de treino

In [76]:
explanatory_features = list(data2.columns)
for i in colunas_para_remover:
    explanatory_features.remove(i)
explanatory_features = data2[explanatory_features]
explanatory_features.shape

(4576, 87)

In [82]:
teste = pd.DataFrame()
for i in explanatory_features.columns:
    if explanatory_features[i].dtype == 'float':
        teste[i]=(explanatory_features[i].fillna(explanatory_features[i].mode().mean())*100).astype(int)
teste.describe()
explanatory_features[teste.columns] = teste[teste.columns] 

C:\Users\1570 LITE\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


* Vamos criar o Data Frame de resposta

In [83]:
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = data2['NU_INSCRICAO']
answer['NU_NOTA_MT']= regr2.predict(explanatory_features)

In [84]:
#norm_zera_neg
y_test_pred2_1 = answer['NU_NOTA_MT'].copy()
y_test_pred2_1[y_test_pred2_1<0] = 0
answer['NU_NOTA_MT'] = y_test_pred2_1/100

<h1>Vamos salvar as respostas como 'answer.csv'</h1>

In [85]:
answer.to_csv('answer.csv',index=0)